# `JuMP.jl`: Beyond the Basics
In the first part of our `JuMP.jl` introduction, we learned how to model/solve simple optimization problems with scalar decision variables. In this part, we take a deeper dive into `JuMP.jl`'s features and to have the tools we need to tackle real problems.

## Resources
Again, for reference, here are resources to learn more and get help:
- The tutorials, examples, manuals, and guides in `JuMP.jl`'s documentation: https://jump.dev/JuMP.jl/stable/
- The Julia optimization forum: https://discourse.julialang.org/c/domain/opt/13
- Julia Programming for Operations Research 2/e (not always up-to-date): https://www.softcover.io/read/7b8eb7d0/juliabook2/introduction

## Models and Solvers
We have learned how to create `Model` objects with an optimizer using its default settings. Now let's take a closer look.

### Solver Specification
As before, we can create by passing an optimizer:

In [3]:
using JuMP, HiGHS

model = Model(HiGHS.Optimizer)

A JuMP Model
Feasibility problem with:
Variables: 0
Model mode: AUTOMATIC
CachingOptimizer state: EMPTY_OPTIMIZER
Solver name: HiGHS

Alternatively, we can create the `Model` add the optimizer later (any time before calling `optimize!`) via `set_optimizer`:

In [4]:
model = Model()
set_optimizer(model, HiGHS.Optimizer)

This all allows us to create a `Model` object and attach a solver to it. However, where possible the optimizer should be provided directly to the `Model` object for better error messaging (e.g., adding a constraint that the solver doesn't support). 

Often times we may wish to specify options (attributes) to the solver. One way to accomplish this is via `optimizer_with_attributes`: 

In [5]:
model = Model(optimizer_with_attributes(HiGHS.Optimizer, "output_flag" => false, "presolve" => "on"))

A JuMP Model
Feasibility problem with:
Variables: 0
Model mode: AUTOMATIC
CachingOptimizer state: EMPTY_OPTIMIZER
Solver name: HiGHS

Here the attributes are solver specific and can be found by checking the documentation associated with each solver. We can also specify/modify attributes using `set_optimizer_attribute`:

In [6]:
model = Model(HiGHS.Optimizer)
set_optimizer_attribute(model, "output_flag", false)
set_optimizer_attribute(model, "presolve", "on")

For convenience, `JuMP.jl` provides a few solver-agnostic methods for setting common attributes such as turning the output off and setting a time limit:

In [7]:
model = Model(HiGHS.Optimizer)
set_silent(model) # turn the output printing off
set_time_limit_sec(model, 60.0) # set a time limit

### File Writing/Reading
`JuMP.jl` does support writing models to files via `write_to_file` and creating a model from a file via `read_from_file`:

In [8]:
write_to_file(model, "model.mps")
read_model = read_from_file("model.mps")

A JuMP Model
Minimization problem with:
Variables: 0
Objective function type: AffExpr
Model mode: AUTOMATIC
CachingOptimizer state: NO_OPTIMIZER
Solver name: No optimizer attached.

For more information on supported files and details about this refer to https://jump.dev/JuMP.jl/stable/manual/models/#Write-a-model-to-file.

### `MathOptInterface.jl` Backends and Performance
More advanced users may wish to better understand what is going on behind the scenes and squeeze out some better performance. This section will touch upon some of these considerations at a surface level. A more throughout discussion is provided at https://jump.dev/JuMP.jl/stable/manual/models/#Backends

`JuMP.jl` `Model`s are thin wrappers `MathOptInterface.jl` models which are what actually store the optimization problem and interface to the solvers. These model(s) employed by `MathOptInterface.jl` behind the scenes are called the backend and can be accessed via `backend`. Let's take a look:

In [9]:
model = Model(HiGHS.Optimizer)
b = backend(model)

MOIU.CachingOptimizer{MOIB.LazyBridgeOptimizer{HiGHS.Optimizer}, MOIU.UniversalFallback{MOIU.Model{Float64}}}
in state EMPTY_OPTIMIZER
in mode AUTOMATIC
with model cache MOIU.UniversalFallback{MOIU.Model{Float64}}
  fallback for MOIU.Model{Float64}
with optimizer MOIB.LazyBridgeOptimizer{HiGHS.Optimizer}
  with 0 variable bridges
  with 0 constraint bridges
  with 0 objective bridges
  with inner model A HiGHS model with 0 columns and 0 rows.

There is quite a bit going on here in addition to the `MOI` `HiGHS.Optimizer` model. We'll briefly describe these different layers. 

The `MOIU.CachingOptimizer` is a layer that allows us to build models incrementally (e.g., adding variables one by one) even if the solver doesn't support that. All the information is stored in `MOIU.UniversalFallback{MOIU.Model{Float64}}` which acts as a cache:

In [10]:
b.model_cache


MOIU.UniversalFallback{MOIU.Model{Float64}}
fallback for MOIU.Model{Float64}

The cache model can always be build/modified incrementally. This is then used to efficiently update the optimizer model `MOIB.LazyBridgeOptimizer{HiGHS.Optimizer}` when appropriate:

In [11]:
b.optimizer

MOIB.LazyBridgeOptimizer{HiGHS.Optimizer}
with 0 variable bridges
with 0 constraint bridges
with 0 objective bridges
with inner model A HiGHS model with 0 columns and 0 rows.

Notice the optimizer model (`HiGHS.Optimizer`) is wrapped in `MOIB.LazyBridgeOptimizer`. The bridge layer allows constraints to be transformed (i.e., bridged) to forms that a solver supports. For example, we might need to split an interval constraint into 2 inequality constraints.

If constraint bridges are unnecessary for our model, and we wish to decrease the start-up latency, we can use `add_bridges = false` when we create the `JuMP.jl` `Model` to eliminate this layer:

In [12]:
model = Model(HiGHS.Optimizer; add_bridges = false)
backend(model)

MOIU.CachingOptimizer{HiGHS.Optimizer, MOIU.UniversalFallback{MOIU.Model{Float64}}}
in state EMPTY_OPTIMIZER
in mode AUTOMATIC
with model cache MOIU.UniversalFallback{MOIU.Model{Float64}}
  fallback for MOIU.Model{Float64}
with optimizer A HiGHS model with 0 columns and 0 rows.

Notice that the `MOIB.LazyBridgeOptimizer` layer on the optimizer is now gone. 

To take this a step further, we can eliminate the cache model entirely by creating a model via `direct_model`:

In [13]:
model = direct_model(HiGHS.Optimizer())
backend(model)

A HiGHS model with 0 columns and 0 rows.

Now we only have the `HiGHS.Optimizer` as the backend model. This avoids the overhead of the caching model which effectively creates two copies of our model. However, there are a few caveats:
- This will not work for solvers that cannot be build incrementally (e.g., Ipopt)
- There are no bridges to reformulate constraints into acceptable forms
- The behavior of querying the solution after modification to the problem is solver specific

So direct models can help increase performance, but should be used carefully.

## Variables
Let's take a deeper dive into more of the things we can do with `@variable`.

### Containers and Sets
We have already seen how to add individual scalar variables, now let's see how to add multiple variables at once.

`JuMP.jl` uses 3 data structures to store variable collections:
- `Array`s: The native Julia arrays
- `DenseAxisArray`s: Dense arrays with arbitrary indices
- `SparseAxisArray`s: Sparse arrays with arbitrary indices

Arrays are created using integer indices of the form `1:n`. For instance, the matrix:

In [14]:
model = Model()
@variable(model, a[1:2, 1:4])

2×4 Matrix{VariableRef}:
 a[1,1]  a[1,2]  a[1,3]  a[1,4]
 a[2,1]  a[2,2]  a[2,3]  a[2,4]

This creates a 2 x 4 matrix of variables that is stored to `a` which we can index and use in defining our problem.

We can also create an n-dimensional vector variable $x \in \mathbb{R}^n$ with upper and lower bounds:

In [15]:
n = 5
l = [1, 2, 3, 4, 5]
u = [10, 11, 12, 13, 14]

@variable(model, l[i] <= x[i = 1:n] <= u[i])

5-element Vector{VariableRef}:
 x[1]
 x[2]
 x[3]
 x[4]
 x[5]

Notice we declare an index `i` to help us define the appropriate values. 

We can use other index forms that don't conform to `1:n` and make `DenseAxisArray`s:

In [16]:
@variable(model, z[i = 2:3, j = 1:2:3] >= i + 2j)

2-dimensional DenseAxisArray{VariableRef,2,...} with index sets:
    Dimension 1, 2:3
    Dimension 2, 1:2:3
And data, a 2×2 Matrix{VariableRef}:
 z[2,1]  z[2,3]
 z[3,1]  z[3,3]

We don't even have to use integers:

In [17]:
@variable(model, w[["red", "blue"], 1:5] <= 1)

2-dimensional DenseAxisArray{VariableRef,2,...} with index sets:
    Dimension 1, ["red", "blue"]
    Dimension 2, Base.OneTo(5)
And data, a 2×5 Matrix{VariableRef}:
 w[red,1]   w[red,2]   w[red,3]   w[red,4]   w[red,5]
 w[blue,1]  w[blue,2]  w[blue,3]  w[blue,4]  w[blue,5]

For indices that do not form a rectangular set, a `SparseAxisArray` is formed:

In [18]:
@variable(model, u[i = 1:2, j = i:3])

JuMP.Containers.SparseAxisArray{VariableRef, 2, Tuple{Int64, Int64}} with 5 entries:
  [1, 1]  =  u[1,1]
  [1, 2]  =  u[1,2]
  [1, 3]  =  u[1,3]
  [2, 2]  =  u[2,2]
  [2, 3]  =  u[2,3]

We can even add a conditional statement after a `;` when defining indices:

In [19]:
@variable(model, v[i = 1:3, j = 1:4; i + j <= 4])

JuMP.Containers.SparseAxisArray{VariableRef, 2, Tuple{Int64, Int64}} with 6 entries:
  [1, 1]  =  v[1,1]
  [1, 2]  =  v[1,2]
  [1, 3]  =  v[1,3]
  [2, 1]  =  v[2,1]
  [2, 2]  =  v[2,2]
  [3, 1]  =  v[3,1]

### Integrality
To specify integer variables, we need only add the `Int` argument:

In [20]:
@variable(model, integer_x, Int)

integer_x

or by setting `integer = true`:

In [21]:
@variable(model, integer_z, integer = true)

integer_z

Similarly, we create binary variables via the `Bin` argument:

In [22]:
@variable(model, binary_x, Bin)

binary_x

or by using `binary = true`:

In [23]:
@variable(model, binary_z, binary = true)

binary_z

### Exercise: Nodal Variables
**Problem**
- Create a variable named `xp`
- `xp` should be integer valued between 0 and 3
- `xp` should be indexed over each arc `(i, j)` in `arcs`

In [24]:
arcs = [(1, 2), (1, 3), (3, 2), (2, 4)]

# PUT CODE HERE
@variable(model, 0 ≤ xp[(i, j) ∈ arcs] ≤ 3)

1-dimensional DenseAxisArray{VariableRef,1,...} with index sets:
    Dimension 1, [(1, 2), (1, 3), (3, 2), (2, 4)]
And data, a 4-element Vector{VariableRef}:
 xp[(1, 2)]
 xp[(1, 3)]
 xp[(3, 2)]
 xp[(2, 4)]

### Other Options
There are a variety of other things we can do with variables. We can create a fixed variable:

In [25]:
@variable(model, x_fixed == 42)

x_fixed

We can specify the initial guess to pass on to the solver via `start`:

In [26]:
@variable(model, q, start = 2)

q

We can also specify lower/upper bounds via the keyword arguments `lower_bound` and `upper_bound`.

### Anonymous Variables
Normally when we create variables they are registered in the model, so we can access them via `model[:var_name]`. For instance, consider a typical definition of a scalar variable:

In [27]:
model = Model()
@variable(model, x >= 0)

x

This does a few things:
- Add a scalar variable with a name `"x"` to `model`
- Add a lower bound of 0
- Create a Julia variable `x` that stores a reference to the optimization variable
- Registers that Julia variable in `model` such that is can be accessed via `model[:x]`

In [28]:
@show x 
@show model[:x];

x = x
model[:x] = x


However, this registration means we cannot add another variable with the same name:

In [29]:
@variable(model, x == 0)

ErrorException: An object of name x is already attached to this model. If this
    is intended, consider using the anonymous construction syntax, for example,
    `x = @variable(model, [1:N], ...)` where the name of the object does
    not appear inside the macro.

    Alternatively, use `unregister(model, :x)` to first unregister
    the existing name from the model. Note that this will not delete the
    object; it will just remove the reference at `model[:x]`.


If this behavior is prohibitive, we can define an anonymous variable:

In [30]:
x = @variable(model, lower_bound = 0, base_name = "x")

x

This adds an optimization variable to the model with a lower bound and name `"x"`, but doesn't register it. We store the resulting variable reference in the Julia variable `x`.

### Modify Variables
There are a variety of ways to change variables after they are created. Some common methods include:
- `set_lower_bound`
- `set_upper_bound`
- `fix`
- `set_start_value`
- `set_binary`
- `set_integer`
- `delete`

For example:

In [31]:
set_upper_bound(x, 10)
set_integer(x)
delete(model, x)

There are many more things we can do, see https://jump.dev/JuMP.jl/stable/manual/variables/ to learn more. 

## Expressions
Sometimes we may want to use a mathematical expression in multiple constraints and/or the objective. We can create expressions using `@expression`. To motivate this, let's create a model with variables:

In [32]:
model = Model()
@variable(model, x[1:2]);

### Expressions
We can create expressions using `@expression`. For instance:

In [33]:
my_expr = @expression(model, x[1]^2 - 3x[2])

x[1]² - 3 x[2]

creates an anonymous expression that we can use elsewhere. We can also create named/registered expressions by adding a name argument:

In [34]:
@expression(model, my_expr, x[1]^2 - 3x[2])
@show my_expr;

my_expr = x[1]² - 3 x[2]


This creates a Julia variable `my_expr` to access the expression and registers it, so we can get it by indexing `model`:

In [35]:
model[:my_expr]

x[1]² - 3 x[2]

We can also create a container of expressions, just like we can for variables:

In [36]:
@expression(model, expr[i = 1:2], 4x[i]^2)

2-element Vector{QuadExpr}:
 4 x[1]²
 4 x[2]²

### Linear Algebra
For linear/quadratic expressions, `JuMP.jl` supports linear algebra operations. For instance, consider $x^T A y$:

In [37]:
@variable(model, y[1:3])
A = [1 2 4; 2 6 1]

@expression(model, x' * A * y)

y[1]*x[1] + 2 y[1]*x[2] + 2 y[2]*x[1] + 6 y[2]*x[2] + 4 y[3]*x[1] + y[3]*x[2]

which is equivalent to:

In [38]:
@expression(model, sum(x[i] * A[i, j] * y[j] for j in 1:3, i in 1:2))

x[1]*y[1] + 2 y[1]*x[2] + 2 y[2]*x[1] + 6 y[2]*x[2] + 4 y[3]*x[1] + y[3]*x[2]

### Nonlinear Expressions
Any nonlinear expressions are also defined via `@expression`. For example:

In [39]:
@expression(model, nlexpr[i = 1:2], 2sin(x[i]))

2-element Vector{NonlinearExpr}:
 2.0 * sin(x[1])
 2.0 * sin(x[2])

The library of built-in univariate operators is derived those listed in `MOI.ListOfSupportedNonlinearOperators`.

In [40]:
import Ipopt

MOI.get(Ipopt.Optimizer(), MOI.ListOfSupportedNonlinearOperators())

86-element Vector{Symbol}:
 :+
 :-
 :abs
 :sign
 :sqrt
 :cbrt
 :abs2
 :inv
 :log
 :log10
 ⋮
 :min
 :max
 :&&
 :||
 :<=
 :(==)
 :>=
 :<
 :>

If we want to use some other nonlinear operator that is not natively supported, we can add our own! For instance, let's add the `logerfcx` from `SpecialFunctions.jl` using `@operator`:

In [41]:
using SpecialFunctions

@operator(model, op_logerfcx, 1, logerfcx) # register a univariate operator `op_logerfcx` and use auto differientiation for gradients
@expression(model, [i = 1:2], op_logerfcx(x[i]))

2-element Vector{NonlinearExpr}:
 op_logerfcx(x[1])
 op_logerfcx(x[2])

For more information on nonlinear expressions see https://jump.dev/JuMP.jl/stable/manual/nonlinear/.

## Objectives
We have already seen how to set linear/quadratic objectives using `@objective`. Let's learn a little more.

### Modification
Once an objective is set, we can modify by simply calling `@objective` again:

In [44]:
# Model to play with
model = Model()
@variable(model, x[1:2])
@variable(model, y[1:3])

# Set objective
@objective(model, Min, log(x[1]) + x[2]^2)
@show objective_function(model)

# Change the objective
@objective(model, Min, 4x[1] + 3x[2])
@show objective_function(model);

objective_function(model) = log(x[1]) + (x[2]²)
objective_function(model) = 4 x[1] + 3 x[2]


If all we want to do is change a linear coefficient, then we can use `set_objective_coefficient` instead:

In [45]:
set_objective_coefficient(model, x[1], 2)
objective_function(model)

2 x[1] + 3 x[2]

### Exercise: Linear Algebra Objective
**Problem**
- Create a quadratic objective
- The function is $x^T A y + b^T y + c^Tx$
- Maximize the objective

In [46]:
A = [1 3 6; -9 2 1]
b = [3, -2, 0]
c = [2, 1]

# PUT CODE HERE
@objective(model, Min, x' * A * y + b' * y + c' * x)

y[1]*x[1] - 9 y[1]*x[2] + 3 y[2]*x[1] + 2 y[2]*x[2] + 6 y[3]*x[1] + y[3]*x[2] + 3 y[1] - 2 y[2] + 2 x[1] + x[2]

## Constraints
Previously, we saw how to add simple scalar constraints. We will now take a deeper dive into using constraints in `JuMP.jl`.

Let's setup a model and variables:

In [47]:
model = Model()
@variable(model, x[1:2])
J = 2:3 # define a set-like index object
@variable(model, y[J]);

### Anonymous Constraints
Commonly, we can declare constraints with names:

In [48]:
@constraint(model, c1, x[1] + 2x[2] >= 42)

c1 : x[1] + 2 x[2] >= 42

This adds a constraint to `model` with the name `"c1"` and creates a Julia variable `c1` that contains a constraint reference which points to the constraint in the model. It also registers the Julia variable such that the constraint can be accessed via `model[:c1]` just like what happens with variables:

In [49]:
@show model[:c1]
@show c1;

model[:c1] = c1 : x[1] + 2 x[2] >= 42
c1 = c1 : x[1] + 2 x[2] >= 42


If we want to, we can omit the name argument and make an anonymous constraint which is not registered:

In [50]:
c1 = @constraint(model, x[1] + 2x[2] >= 42)

x[1] + 2 x[2] >= 42

So we don't have to name every constraint if we don't want to.

### Constraint Abstraction
Constraints in `JuMP` (which are stored in the `MOI` backend) are stored with the form `function` in `set`. Here `function` can be any scalar/vector-valued algebraic expression and `set` describes the constraint placed on the expression. For instance, let's consider the linear constraint $x_1 + 2x_2 \geq 42$:

In [51]:
@constraint(model, c2, x[1] + 2x[2] >= 42)

c2 : x[1] + 2 x[2] >= 42

Let's interrogate the `function` and the `set`:

In [52]:
raw_constr = constraint_object(c1)
@show jump_function(raw_constr)
@show moi_set(raw_constr);

jump_function(raw_constr) = x[1] + 2 x[2]
moi_set(raw_constr) = MathOptInterface.GreaterThan{Float64}(42.0)


So, we have a linear expression $x_1 + 2x_2$ and a constraint set $\geq 42$ which constrains the expression to be greater than 42. If we were so inclined, we could directly express the constraint this way:

In [53]:
@constraint(model, x[1] + 2x[2] in MOI.GreaterThan(42.0))

x[1] + 2 x[2] >= 42

To learn more about all the sets that `MOI` supports see https://jump.dev/JuMP.jl/stable/moi/manual/constraints/#Constraints-by-function-set-pairs. We will keep the remainder of the discussion to the symbolic forms that `JuMP` provides which conveniently wrap around these underlying sets.

A key consequence of this modeling abstraction is that `JuMP` *normalizes* constraints, moving variables to the right-hand side and moving constants to the left-hand side. For instance:

In [54]:
@constraint(model, 2x[1] + 1 <= 4x[1] + 4)

-2 x[1] <= 3

### Constraint Senses
Here we review the symbolic senses supported by `@constraint`. We illustrate these below:

In [55]:
@constraint(model, 4 <= 2 * x[2] <= 5)            # `lb <= expr <= ub` interval     (can also use `≤`)
@constraint(model, sum(x) <= 1)                   # `<=`               less than    (can also use `≤`)
@constraint(model, x[1] + 2 * x[2] >= 2)          # `>=`               greater than (can also use `≥`)
@constraint(model, sum(j * y[j] for j in J) == 3) # `==`               equal to

2 y[2] + 3 y[3] == 3

We can also use the vectorized version of these operators by adding a `.` in front of the operator. This is often useful with linear algebra definitions:

In [56]:
A = [1 2; 3 4]
b = [5, 6]

@constraint(model, A * x .== b)

2-element Vector{ConstraintRef{Model, MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64}, MathOptInterface.EqualTo{Float64}}, ScalarShape}}:
 x[1] + 2 x[2] == 5
 3 x[1] + 4 x[2] == 6

This can help write some very compact formulations.

### Using Sets/Containers
In similar manner to expressions and variables, we can create collections of constraints using `JuMP` containers. For instance, consider the constraint $x_i^2 + 4y_j \leq 0, \ i \in \{1, 2\}, j \in J$:

In [57]:
@constraint(model, my_constr[i ∈ 1:2, j ∈ J], x[i]^2 + 4y[j] ≤ 0)

2-dimensional DenseAxisArray{ConstraintRef{Model, MathOptInterface.ConstraintIndex{MathOptInterface.ScalarQuadraticFunction{Float64}, MathOptInterface.LessThan{Float64}}, ScalarShape},2,...} with index sets:
    Dimension 1, Base.OneTo(2)
    Dimension 2, 2:3
And data, a 2×2 Matrix{ConstraintRef{Model, MathOptInterface.ConstraintIndex{MathOptInterface.ScalarQuadraticFunction{Float64}, MathOptInterface.LessThan{Float64}}, ScalarShape}}:
 my_constr[1,2] : x[1]² + 4 y[2] <= 0  my_constr[1,3] : x[1]² + 4 y[3] <= 0
 my_constr[2,2] : x[2]² + 4 y[2] <= 0  my_constr[2,3] : x[2]² + 4 y[3] <= 0

Note that the name `my_constr` is optional and instead of `in` we used `∈`. Here the supported index syntax is exactly the same as `@variable`, in fact all the `JuMP` macros use the same syntax. We can access individual constraints by indexing the container we generate:

In [58]:
my_constr[2, 3]

my_constr[2,3] : x[2]² + 4 y[3] <= 0

This syntax is very flexible and can accommodate a wide variety of indexing schemes in native Julia code.

### Exercise: Arc Constraints
**Problem**
- Define constraints of form $2x_i + y_j = 0, \ (i, j) \in A$

In [59]:
A = [(1, 2), (2, 3), (2, 2)]

# PUT CODE HERE
@constraint(model, [(i, j) ∈ A], 2x[i] + y[j] == 0)

1-dimensional DenseAxisArray{ConstraintRef{Model, MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64}, MathOptInterface.EqualTo{Float64}}, ScalarShape},1,...} with index sets:
    Dimension 1, [(1, 2), (2, 3), (2, 2)]
And data, a 3-element Vector{ConstraintRef{Model, MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64}, MathOptInterface.EqualTo{Float64}}, ScalarShape}}:
 2 x[1] + y[2] == 0
 2 x[2] + y[3] == 0
 2 x[2] + y[2] == 0

### Modification
That are several ways to modify constraints. We will highlight a few.

Recall that all constants are moved to the RHS. We can modify the RHS of a constraint via `set_normalized_rhs`:


In [60]:
con = @constraint(model, 2x[1] <= 1)
@show con 
set_normalized_rhs(con, 3)
@show con;

con = 2 x[1] <= 1
con = 2 x[1] <= 3


We can also change the coefficient of a linear variable via `set_normalized_coefficient`:

In [61]:
@show con 
set_normalized_coefficient(con, x[1], -1)
@show con;

con = 2 x[1] <= 3
con = -x[1] <= 3


We can also delete constraints via `delete`:

In [62]:
delete(model, con)

### Other Constraints
We will now highlight other constraint types that are natively supported by `JuMP`.

First, consider second-order cone constraints $||x||_2 \leq t$:

In [63]:
model = Model()
@variable(model, t)
@variable(model, x[1:2])
@constraint(model, [t; x] in SecondOrderCone())

[t, x[1], x[2]] in MathOptInterface.SecondOrderCone(3)

Next, rotated second order cone constraints $||x||_2^2 \leq 2t \cdot u$:

In [64]:
model = Model()
@variable(model, t)
@variable(model, u)
@variable(model, x[1:2])
@constraint(model, [t; u; x] in RotatedSecondOrderCone())

[t, u, x[1], x[2]] in MathOptInterface.RotatedSecondOrderCone(4)

Next, semi-continuous variables $y \in \{0\} \cup [l, u]$ and semi-integer variables $z \in \{0\} \cup [l, l + 1, \dots, u]$:

In [65]:
@variable(model, y)
@constraint(model, y in MOI.Semicontinuous(1.5, 3.5))
@variable(model, z)
@constraint(model, z in MOI.Semiinteger(1.0, 3.0))

z in MathOptInterface.Semiinteger{Float64}(1.0, 3.0)

Next, special ordered sets of type 1 (SOS1) and SOS2 constraints:

In [66]:
@variable(model, v[1:3])
@constraint(model, v in SOS1())
@constraint(model, v in SOS2())

[v[1], v[2], v[3]] in MathOptInterface.SOS2{Float64}([1.0, 2.0, 3.0])

Next, indicator constraints where a linear constraint is enforced when a binary variable is 1:

In [67]:
@variable(model, a, Bin)
@constraint(model, a => {y + z <= 1})
@constraint(model, !a => {z >= 3}) # inverted logic

!a --> {z >= 3}

Next, positive-semi definite (PSD) constraints:

In [68]:
@variable(model, X[1:2, 1:2])
@constraint(model, X >= 0, PSDCone()) # note it is preferred to define as `@variable(X[1:2, 1:2], PSD)`

[X[1,1]  X[1,2]
 X[2,1]  X[2,2]] in PSDCone()

Finally, we'll mention complementarity constraints $F(s) \perp s$ with $s \in [lb, ub]$:

In [69]:
@variable(model, 0 <= s <= 1)
@constraint(model, 2s - 1 ⟂ s)

[2 s - 1, s] in MathOptInterface.Complements(2)

I will also note that constraint programming constraints are also supported: https://jump.dev/JuMP.jl/stable/tutorials/linear/constraint_programming/.

## Solutions
We have already reviewed the common query methods which include:
- `termination_status`
- `primal_status`
- `dual_status`
- `objective_value`
- `value`
- `shadow_price`

Here we will take closer look and review a few more of the available methods.

Let's first setup an optimized model that we can query:

In [70]:
model = Model(HiGHS.Optimizer)
set_silent(model)
@variable(model, x >= 0)
@variable(model, y[[:a, :b]] <= 1)
@objective(model, Max, -12x - 20y[:a])
@expression(model, my_expr, 6x + 8y[:a])
@constraint(model, my_expr >= 100)
@constraint(model, c1, 7x + 12y[:a] >= 120)
optimize!(model)

### Solution Summary
For a general overview, we can use `solution_summary`:

In [71]:
solution_summary(model)

* Solver : HiGHS

* Status
  Result count       : 1
  Termination status : OPTIMAL
  Message from the solver:
  "kHighsModelStatusOptimal"

* Candidate solution (result #1)
  Primal status      : FEASIBLE_POINT
  Dual status        : FEASIBLE_POINT
  Objective value    : -2.05143e+02
  Objective bound    : -0.00000e+00
  Relative gap       : Inf
  Dual objective value : -2.05143e+02

* Work counters
  Solve time (sec)   : 9.98020e-04
  Simplex iterations : 2
  Barrier iterations : 0
  Node count         : -1


We can get even more information if we wish:

In [72]:
solution_summary(model, verbose=true)

* Solver : HiGHS

* Status
  Result count       : 1
  Termination status : OPTIMAL
  Message from the solver:
  "kHighsModelStatusOptimal"

* Candidate solution (result #1)
  Primal status      : FEASIBLE_POINT
  Dual status        : FEASIBLE_POINT
  Objective value    : -2.05143e+02
  Objective bound    : -0.00000e+00
  Relative gap       : Inf
  Dual objective value : -2.05143e+02
  Primal solution :
    x : 1.54286e+01
    y[a] : 1.00000e+00
    y[b] : 1.00000e+00
  Dual solution :
    c1 : 1.71429e+00

* Work counters
  Solve time (sec)   : 9.98020e-04
  Simplex iterations : 2
  Barrier iterations : 0
  Node count         : -1


### Termination Status
We already discussed querying the statuses which are independent of the solver used. We can also extract the raw status as report by the solver via `raw_status`:

In [73]:
raw_status(model)

"kHighsModelStatusOptimal"

### Primal Solutions
Before querying values, we should always check that there are some we can actually get via `has_values`:

In [74]:
has_values(model)

true

To query the value of a container of a variable/expression/constraint collection, we broadcast over `value`:

In [75]:
value.(y)

1-dimensional DenseAxisArray{Float64,1,...} with index sets:
    Dimension 1, [:a, :b]
And data, a 2-element Vector{Float64}:
 1.0
 1.0

This returns a container with the same indices that contains the optimal values.

### Dual Solutions
We can check if there are duals to query via `has_duals`:

In [76]:
has_duals(model)

true

We can get the dual objective value via `dual_objective_value`:

In [77]:
dual_objective_value(model)

-205.1428571428571

We can get the dual solution via `dual`:

In [78]:
dual(c1)

1.7142857142857142

Or get the duals of the variable bound via `LowerBoundRef`, `UpperBoundRef`, or `FixRef`:

In [79]:
@show dual(LowerBoundRef(x))
@show dual.(UpperBoundRef.(y));

dual(LowerBoundRef(x)) = 0.0
dual.(UpperBoundRef.(y)) = 1-dimensional DenseAxisArray{Float64,1,...} with index sets:
    Dimension 1, [:a, :b]
And data, a 2-element Vector{Float64}:
 -0.5714285714285694
  0.0


We should note that `JuMP`'s definition of dual depends on the constraint direction, not the objective sense (different from some linear programming conventions). If we want the other convention, we can use `shadow_price` and `reduced_cost` instead:

In [80]:
@show shadow_price(c1)
@show reduced_cost(x)
@show reduced_cost.(y);

shadow_price(c1) = 1.7142857142857142
reduced_cost(x) = -0.0
reduced_cost.(y) = 1-dimensional DenseAxisArray{Float64,1,...} with index sets:
    Dimension 1, [:a, :b]
And data, a 2-element Vector{Float64}:
  0.5714285714285694
 -0.0


### Other Queries
Some other attributes we can query are:

In [81]:
@show solve_time(model)
@show relative_gap(model)
@show simplex_iterations(model)
@show barrier_iterations(model)
@show node_count(model);

solve_time(model) = 0.0009980201721191406
relative_gap(model) = Inf
simplex_iterations(model) = 2
barrier_iterations(model) = 0
node_count(model) = -1


Some other things we can do which are beyond the scope of today include:
- Linear sensitivity analysis via `lp_sensitivity_report`
- Conflict analysis for infeasible models via `compute_conflict!`
- Feasibility checking via `primal_feasibility_report`
- For solver that return multiple solutions, we can use the `result` keyword to get the one we want

For more information see https://jump.dev/JuMP.jl/stable/manual/solutions/.

## Other Functionalities
Without going into too much detail we'll mention some other things `JuMP` can do.

### Plural Macros
Instead of having many individual calls to macros like `@variable`/`@constraint`, we can use the plural version by adding an `s` at the end. For example, for variables we can use `@variables`:

In [82]:
model = Model()

@variables(model, begin
    x
    y[i=1:2] >= i, (start = i, base_name = "Y_$i")
    z, Bin
end)

latex_formulation(model)

$$ \begin{aligned}
\text{feasibility}\\
\text{Subject to} \quad & Y\_1_{1} \geq 1\\
 & Y\_2_{2} \geq 2\\
 & z \in \{0, 1\}\\
\end{aligned} $$

### Solver-Independent Callbacks
Callbacks can be powerful ways to modify the way optimization problems are solved. Typically, this is solver dependent, but `JuMP` provides a solver-independent API. In particular, three types of callbacks are supported:
- lazy constraints
- user-cuts
- heuristic solutions

Note that this is only supported with a few solvers such as CPLEX, GLPK, Gurobi, and Xpress. For details, see https://jump.dev/JuMP.jl/stable/manual/callbacks/.

### Parameters
Declaring parameters can a useful way to way the values of constants in a model without having reconstruct the whole thing. This can be accomplished via the `Parameter` set:

In [83]:
@variable(model, p[i = 1:2] in Parameter(i))


2-element Vector{VariableRef}:
 p[1]
 p[2]

We can query and update the values via `parameter_value` and `set_parameter_value`:

In [84]:
@show parameter_value.(p)

set_parameter_value(p[2], 3.0)

@show parameter_value.(p);

parameter_value.(p) = [1.0, 2.0]
parameter_value.(p) = [1.0, 3.0]


We can use these in any expression/objective/constraint:

In [85]:
@objective(model, Max, p[1] * x)
@expression(model, my_nl_expr, p[1] * x^2)


p[1] * (x²)

For affine/quadratic expressions/objectives/constraints, we have a few options.

First, we can use:
- `set_objective_coefficient`
- `set_normalized_rhs`
- `set_normalized_coefficient`

which we have already discussed. 

## Extensions
To add to the capabilities of `JuMP`, there are a variety of extension packages:
- `StochasticPrograms.jl`: Solve 2-stage stochastic programs
- `BilevelJuMP.jl`: Solve bi-level optimization problems
- `Coluna.jl`: Implement branch-and-price-and-cut approaches
- `Plasmo.jl`: Solve/decompose graph optimization models
- `PolyJuMP.jl`: Solve polynomial optimization problems
- `SDDP.jl`: Solve multi-stage stochastic problems via SDDP
- `SumOfSquares.jl`: Solve polynomial optimization problems
- `vOptGeneric.jl`: Multi-objective optimization
- `InfiniteOpt.jl`: Solve infinite-dimensional optimization problems
- `DisjunctiveProgramming.jl`: Solve GDP problems

We'll focus today on `InfiniteOpt.jl`!